## 4-4 DQN을 활용한 카트-폴
### 4-4-1 DQN을 활용한 카트-폴
`DQN (Deep Q-Network)`에서는 테이블 형식이 아닌 뉴럴 네트워크로 행동 가치 함수를 표현한다. 입력은 '상태', 출력은 '행동'이 되는 뉴럴 네트워크로 특정한 상태에서 특정한 행동을 선택할 확률을 추론한다. <br>
DQN 논문은 <br>
[Playing Atari with Deep Reinforcement Learning](https://arxiv.org/abs/1312.5602)<br>
[Human-level control through deep reinforcement learning](https://nature.com/articles/nature14236)<br>
에서 확인 가능하다. <br>
<br>
이번에는 DQN으로 `OpenAI Gym`환경 중 하나인 `Cart-Pole`을 공략한다. 막대를 쓰러뜨리지 않고 균형을 잡는 게임이다. <br>
보상의 경우 에피소드 완료 시 190 스텝 이상이면 +1, 상태는 카트의 위치, 속도, 막대의 각도, 각속도이다. 

### 4-4-2 뉴럴 네트워크 입력과 출력
테이블 형식에서는 행동 가치 함수가 Q 학습에서의 갱신 계산식을 통해 갱신했지만, 뉴럴 네트워크는 뉴럴 네트워크의 학습에 따라 갱신한다. <br>
뉴럴 네트워크의 입력은 환경의 상태다. 카트-폴의 상태의 수는 4이므로, 입력 형태는 (4,)이다. <br>
뉴럴 네트워크의 출력은 행동별 가치다. 카트-폴에서의 행동의 수는 2이므로, 출력 형태는 (2,)이다. <br>
이 뉴럴 네트워크를 학습함에 따라 특정 상태에서 특정 행동을 선택할 때의 가치를 추론할 수 있게 된다.

### 4-4-3 DQN의 4가지 기반 기술
DQN은 Q 학습에서의 행동 가치 함수를 단순히 뉴럴 네트워크로 변경한 것만이 아니라, 안정된 학습을 위해 다음의 4가지 기반 기술을 표현한다.

- Experience Replay

Q 학습에서는 경험(상태, 행동, 보상, 다음 상태)을 순서에 따라 학습했다. 이 방법으로는 시간적으로 상관 관계가 높은 내용을 연속해서 학습하므로 학습이 안정되지 않을 수 있다. <br>
DQN에서는 기억에 경험을 많이 저장한 뒤, 나중에 무작위로 학습한다. 이를 `Experience Replay`라고 한다.

- Fixed Target Q-Network

Q 학습에서는 행동 가치 함수 자체를 이용해 행동 가치 함수를 갱신했다. 즉, 갱신 중인 뉴럴 네트워크를 사용해서 해당 뉴럴 네트워크 갱신을 위한 계산을 하게 됨에 따라 학습이 안정되지 않을 수 있다. <br>
DQN에서는 갱신량만을 계산하는 별도의 뉴럴 네트워크를 사용해서 이 문제를 해결했다. 이를 `Fixed Target Q-Network`라고 한다. <br>
갱신 대상이 되는 뉴럴 네트워크를 `메인 네트워크`, 갱신량을 계산하기 위한 뉴럴 네트워크를 `대상 네트워크`라고 부른다. 대상 네트워크는 과거의 메인 네트워크로 일정 간격으로 메인 네트워크의 가중치를 대상 네트워크에 덮어쓰면서 갱신한다.

- Reward Clipping

환경으로부터 얻는 보상은 환경에 따라 그 스케일이 다르다. 이를 보완하기 위해 모든 환경에서의 보상 스케일을 -1, 0, 1로 고정한다. 이를 `Reward Clipping`이라고 부른다. <br>
이를 통해 환경에 관계없이 동일한 하이퍼 파라미터를 사용해 학습을 수행할 수 있다.

- Huber Loss

뉴럴 네트워크의 오차가 큰 경우에는 오차 함수로 평균 제곱 오차를 사용하면 출력이 너무 커져 학습이 안정되지 않을 수 있다. <br>
DQN에서는 오차가 큰 경우에도 값이 안정되도록 `휴버 함수`를 사용한다.

### 4-4-4 패키지 임포트

In [17]:
# 패키지 임포트
import gym
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from collections import deque
from tensorflow.losses import huber_loss

### 4-4-5 파라미터 준비

In [6]:
# 파라미터 준비
NUM_EPISODES = 500 # 에피소드 수
MAX_STEPS = 200 # 최대 스텝 수
GAMMA = 0.99 # 시간 할인율
WARMUP = 10 # 초기화 시 조작하지 않는 스텝 수

# 검색 파라미터
E_START = 1.0 # epsilon 초기화
E_STOP = 0.01 # epsilon 최종값
E_DECAY_RATE = 0.001 # epsilon 감쇠율

# 메모리 파라미터
MEMORY_SIZE = 10000 # 경험 메모리 사이즈
BATCH_SIZE = 32 # 배치 사이즈

### 4-4-6 행동 평가 함수 정의
행동 평가 함수가 되는 뉴럴 네트워크 모델을 생성한다.

In [18]:
# 행동 평가 함수 정의
class QNetwork:
    # 초기화
    def __init__(self, state_size, action_size):
        # 모델 생성
        self.model = Sequential()
        self.model.add(Dense(16, activation='relu', input_dim=state_size))
        self.model.add(Dense(16, activation='relu'))
        self.model.add(Dense(16, activation='relu'))
        self.model.add(Dense(action_size, activation='linear'))
        
        # 모델 컴파일
        self.model.compile(loss=huber_loss, optimizer=Adam(lr=0.001))

### 4-4-7 경험 메모리 정의
`경험 메모리`는 과거의 경험(상태, 행동, 보상, 다음 상태)를 저장하는 메모리다. <br>
매 스텝마다 add()로 경험을 추가하고, sample()로 배치 사이즈만큼의 경험을 랜덤으로 취득해서 뉴럴 네트워크의 학습을 수행한다. <br>
경험 메모리의 경험은 deque에 저장된다. deque는 list()와 비슷한 데이터 타입이나, 인수 maxlen 이상의 엘리먼트를 추가하려는 처음부터 삭제한다.

In [9]:
# 경험 메모리 정의
class Memory():
    # 초기화
    def __init__(self, memory_size):
        self.buffer = deque(maxlen=memory_size)
        
    # 경험 추가
    def add(self, experience):
        self.buffer.append(experience)
        
    # 배치 사이즈만큼의 경험을 랜덤으로 얻음
    def sample(self, batch_size):
        idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
        return [self.buffer[i] for i in idx]
    
    # 경험 메모리 사이즈
    def __len__(self):
        return len(self.buffer)

### 4-4-8 환경 생성
Env는 gym.make()로 생성한다. 여기서는 카트-폴의 환경을 이용하므로 인수에 `CartPole-v0`을 지정한다.

In [14]:
# 환경 생성
env = gym.make('CartPole-v0')
state_size = env.observation_space.shape[0] # 행동 수
action_size = env.action_space.n # 상태 수

### 4-4-9 메인 네트워크, 대상 네트워크 및 경험 메모리 생성
앞서 정의한 Q-Network와 Memory를 이용해 메인 네트워크, 대상 네트워크, 그리고 경험 메모리를 생성한다.

In [19]:
# 메인 네트워크
main_qn = QNetwork(state_size, action_size)

# 대상 네트워크 생성
target_qn = QNetwork(state_size, action_size)

# 경험 메모리 생성
memory = Memory(MEMORY_SIZE)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


### 4-4-10 학습 시작
- 환경 초기화

Env의 reset()을 이용하여 환경을 초기화한다. 반환값 상태의 형태를 모델에 전달할 데이터 타입에 맞춰 '[4](상태 수) $\rightarrow$ [1, 4](배치 크기, 상태 수)'로 변환한다.

- 에피소드 수만큼 에피소드를 반복

- 대상 네트워크 갱신

에피소드마다 메인 네트워크의 가중치를 대상 네트워크에 덮어쓰는 처리를 수행한다. 가중치 덮어쓰기는 메인 네트워크 model의 get_weights()와 대상 네트워크 model의 set_weights()로 수행한다.

- 1 에피소드의 루프

- 에피소드 완료 시 로그 표시

1 에피소드 완료 후 에피소드 번호, 스텝 수, $\epsilon$의 로그를 표시한다. 그 후, 5회 연속 성공하면 학습 완료, 그렇지 않으면 다음 에피소드를 수행하기 위해 '환경 초기화'를 수행한다.

In [23]:
# 학습 시작

# 환경 초기화
state = env.reset()
state = np.reshape(state, [1, state_size])

# 에피소드 수만큼 에피소드 반복
total_step = 0 # 총 스텝 수
success_count = 0 # 성공 수
for episode in range(1, NUM_EPISODES + 1):
    step = 0 # 스텝 수
    
    # 대상 네트워크 갱신
    target_qn.model.set_weights(main_qn.model.get_weights())
    
    # 1 에피소드 루프
    for _ in range(1, MAX_STEPS + 1):
        step += 1
        total_step += 1
        
        # epsilon 감소시킴
        epsilon = E_STOP + (E_START - E_STOP) * np.exp(-E_DECAY_RATE * total_step)
        
        # 랜덤하게 행동 선택
        if epsilon > np.random.rand():
            action = env.action_space.sample()
        # 행동 가치 함수에 따른 행동 선택
        else:
            action = np.argmax(main_qn.model.predict(state)[0])
            
        # 행동에 맞추어 상태와 보상을 얻음
        next_state, _, done, _ = env.step(action)
        next_state = np.reshape(next_state, [1, state_size])
        
        # 에피소드 완료 시
        if done:
            # 보상 지정
            if step >= 190:
                success_count += 1
                reward = 1
            else:
                success_count = 0
                reward = 0
            
            # 다음 상태에 상태 없음을 대입
            next_state = np.zeros(state.shape)
            
            # 경험 추가
            if step > WARMUP:
                memory.add((state, action, reward, next_state))
        
        # 에피소드 미완료 시
        else:
            # 보상 지정
            reward = 0
            
            # 경험 추가
            if step > WARMUP:
                memory.add((state, action, reward, next_state))
                
            # 상태에 다음 상태 대입
            state = next_state
            
        # 행동 평가 함수 갱신
        if len(memory) >= BATCH_SIZE:
            # 뉴럴 네트워크의 입력과 출력 준비
            inputs = np.zeros((BATCH_SIZE, 4)) # 입력(상태)
            targets = np.zeros((BATCH_SIZE, 2)) # 출력(행동별 가치)
            
            # 배치 사이즈만큼 경험을 랜덤하게 선택
            minibatch = memory.sample(BATCH_SIZE)
            
            # 뉴럴 네트워크 입력과 출력 생성
            for i, (state_b, action_b, reward_b, next_state_b) in enumerate(minibatch):
                
                # 입력 상태 지정
                inputs[i] = state_b
                
                # 선택한 행동의 가치 계산
                if not (next_state_b == np.zeros(state_b.shape)).all(axis=1):
                    target = reward_b + GAMMA * np.amax(target_qn.model.predict(next_state_b)[0])
                else:
                    target = reward_b
                    
                # 출력에 행동별 가치 지정
                targets[i] = main_qn.model.predict(state_b)
                targets[i][action_b] = target # 선택한 행동 가치
                
            # 행동 가치 함수 갱신
            main_qn.model.fit(inputs, targets, epochs=1, verbose=0)
            
        # 에피소드 완료 시
        if done:
            # 에피소드 루프 이탈
            break
    
    # 에피소드 완료 시 로그 표시
    print('에피소드: {}, 스텝 수: {}, epsilon: {:.4f}'.format(episode, step, epsilon))
    
    # 5회 연속 성공으로 학습 완료
    if success_count >= 5:
        break
        
    # 환경 초기화
    state = env.reset()
    state = np.reshape(state, [1, state_size])

에피소드: 1, 스텝 수: 18, epsilon: 0.9823
에피소드: 2, 스텝 수: 23, epsilon: 0.9602
에피소드: 3, 스텝 수: 47, epsilon: 0.9166
에피소드: 4, 스텝 수: 15, epsilon: 0.9031
에피소드: 5, 스텝 수: 19, epsilon: 0.8863
에피소드: 6, 스텝 수: 51, epsilon: 0.8427
에피소드: 7, 스텝 수: 14, epsilon: 0.8311
에피소드: 8, 스텝 수: 20, epsilon: 0.8149
에피소드: 9, 스텝 수: 13, epsilon: 0.8045
에피소드: 10, 스텝 수: 28, epsilon: 0.7826
에피소드: 11, 스텝 수: 21, epsilon: 0.7665
에피소드: 12, 스텝 수: 23, epsilon: 0.7493
에피소드: 13, 스텝 수: 14, epsilon: 0.7390
에피소드: 14, 스텝 수: 19, epsilon: 0.7253
에피소드: 15, 스텝 수: 41, epsilon: 0.6966
에피소드: 16, 스텝 수: 17, epsilon: 0.6850
에피소드: 17, 스텝 수: 51, epsilon: 0.6514
에피소드: 18, 스텝 수: 98, epsilon: 0.5916
에피소드: 19, 스텝 수: 76, epsilon: 0.5490
에피소드: 20, 스텝 수: 177, epsilon: 0.4616
에피소드: 21, 스텝 수: 36, epsilon: 0.4456
에피소드: 22, 스텝 수: 117, epsilon: 0.3975
에피소드: 23, 스텝 수: 23, epsilon: 0.3887
에피소드: 24, 스텝 수: 38, epsilon: 0.3746
에피소드: 25, 스텝 수: 200, epsilon: 0.3085
에피소드: 26, 스텝 수: 200, epsilon: 0.2544
에피소드: 27, 스텝 수: 200, epsilon: 0.2101
에피소드: 28, 스텝 수: 200, epsilon: 0.

### 4-4-11 1 에피소드 루프
1 에피소드만큼 게임 종료까지의 처리를 수행한다.

- $\epsilon$ 감소

파라미터 E_STOP, E_START, E_DECAY_RATE에 맞추어 $\epsilon$을 감소시킨다.

- 무작위 또는 행동 가치 함수에 따라 행동 취득

$\epsilon$과 난수에 맞추어 랜덤으로 또는 행동 가치 함수에 따라 행동을 선택한다. 랜덤 행동은 `env.action_space.sample()`, 행동 가치 함수에 따른 행동은 `np.argmax(main_qn.model.predict(state)[0])`으로 얻는다.

- 행동에 맞춰 상태와 보상을 얻음

Env의 step()을 사용해 행동에 맞추어 상태와 에피소드 완료 여부를 얻는다. <br>
카트-폴이 제공하는 reward도 취득할 수 있지만, 독자적인 보상(다음 에피소드 완료 시 보상)을 사용한다.

- 에피소드 완료 시 처리

에피소드 완료 시 190 스텝 이상이면 보상 및 성공 횟수에 1을 더한다. <br>
그리고 다음 상태에 `상태 없음(0만 있는 배열)`을 대입하고, 경험 메모리에 `경험`을 추가한다. 경험은 스텝 수가 `WARMUP` 이상인 경우에만 추가한다.

- 에피소드 완료 불가 시 처리

에피소드가 완료되지 않는 경우, 즉 보통의 스텝 실행 시에는 보상으로 0을 지정한다.

### 4-4-12 행동 가치 함수 갱신
경험 메모리 수가 배치 사이즈 이상인 경우, 행동 가치 함수인 메인 네트워크를 갱신한다. 

1. 뉴럴 네트워크 입력과 출력 준비

뉴럴 네트워크의 입력 `inputs`와 출력 `targets`를 준비한다. 입력은 '상태', 형태는 '(배치 사이즈, 4)' 출력은 '행동별 가치', 형태는 '(배치 사이즈, 2)'이다.

2. 배치 사이즈만큼 경험을 랜덤으로 취득

3. 뉴럴 네트워크의 입력과 출력 생성

얻어낸 경험으로부터 상태(state_b), 행동(action_b), 보상(reward_b), 다음 상태(next_state_b)를 1 세트씩 꺼낸다. 이 값을 이용해 뉴럴 네트워크의 입력과 출력의 내용을 생성한다. <br>
입력에 상태를 지정하기 위해 inputs[i]에 'state_b'를 대입한다. <br>
선택한 행동의 가치를 계산한다. 다음 행동이 존재하지 않는 경우에는 target에 'reward_b'를 대입하고, 다음 행동이 존재하는 경우에는 다음 식으로 가치를 대입한다. <br>
$$Q(s_t, a_t) \leftarrow R_{t+1} + gamma \times \max_{a}Q(s_{t+1}, a)$$
출력에 행동 별 가치를 지정하기 위해 targets[i]에 행동 가치 함수 메인 네트워크에서 추론한 '행동별 가치'를 대입한다. 그리고 targets[i][action_b]에 앞서 계산한 선택한 행동의 가치를 대입한다.

4. 행동 가치 함수 갱신

생성한 뉴럴 네트워크의 입력과 출력을 사용해 행동 가치 함수 메인 네트워크를 학습시킨다.

### 4-4-13 에피소드 완료 시
### 4-4-14 학습 실행 결과
### 4-4-15 디스플레이 설정
로커렝서는 스텝마다 Env의 `render()`를 호출해서 다른 윈도우 화면에서 환경을 표시할 수 있다. <br>
구글 Colab과 같이 클라우드에서 실행하는 경우에는 화면을 표시할 수 없으며, 에러가 발생한다. <br>
Xvfb(X virtual framebuffer)는 X 윈도우 시스템의 가상 디스플레이를 만들어 주는 소프트웨어다. 이를 활용해 실제 스크린이 없는 상황에서 GUI를 이용해 프로그램을 실행할 수 있다. <br>
`pyvirtualdisplay`는 파이썬에서 가상 디스플레이(Xvfb)를 생성하는 패키지다.

In [ ]:
# 디스플레이 설정 install
#apt-get -qq -y install xvfb freeglut3-dev ffmpeg x11-utils > /dev/null
!pip install pyglet==1.3.2
!pip install pyopengl
!pip install pyvirtualdisplay

# 디스플레이 설정 적용
from virtualdisplay import Display
import os
disp = Display(visible=0, size=(1024, 768))
disp.start()
os.environ['DISPLAY'] = ':' + str(disp.display) + '.' + str(disp.screen)

### 4-4-16 애니메이션 프레임 생성
1 에피소드만큼 게임을 실행해서 스텝 수만큼의 화면 이미지를 수집한다. Env의 `render(mode='rgb_array')`를 호출해 화면의 이미지를 취득할 수 있다.

In [26]:
# 평가
frames = [] # 애니메이션 프레임

# 환경 초기화
state = env.reset()
state = np.reshape(state, [1, state_size])

# 1 에피소드 루프
step = 0 # 스텝 수
for _ in range(1, MAX_STEPS + 1):
    step += 1
    
    # 애니메이션 프레임 추가
    frames.append(env.render(mode='rgb_array'))
    
    # 최적 행동 선택
    action = np.argmax(main_qn.model.predict(state)[0])
    
    # 행동에 맞추어 상황과 보상을 얻음
    next_state, reward, done, _ = env.step(action)
    next_state = np.reshape(next_state, [1, state_size])
    
    # 에피소드 완료 시
    if done:
        # 다음 상태에서 상태 없음을 대입
        next_state = np.zeros(state.shape)
        
        # 에피소드 루프 이탈
        break
    else:
        # 상태에 다음 상태를 대입
        state = next_state
        
# 에피소드 완료 시 로그 표시
print('스텝 수 : {}'.format(step))

ImportError: 
    Cannot import pyglet.
    HINT: you can install pyglet directly via 'pip install pyglet'.
    But if you really just want to install all Gym dependencies and not have to think about it,
    'pip install -e .[all]' or 'pip install gym[all]' will do it.
    

### 4-4-17 애니메이션 프레임을 애니메이션으로 변환
애니메이션 프레임을 애니메이션으로 변환하는 경우에는 `JSAnimation`을 사용한다. matplotlib에서 javascript 애니메이션을 생성하는 패키지다.<br>
애니메이션을 관리하는 `FuncAnimation` 객체를 생성한다. 인수로 figure 객체, 애니메이션 정기 처리, 프레임 수 및 1 프레임의 시간을 지정한다. 객체는 plt.gcf()로 얻어낸다. <br>
이를 변경함으로써 애니메이션 표시 내용도 변경된다. 정기 처리에서는 frames의 이미지를 순서대로 표시한다.<br>
그리고 display_animation으로 HTML 객체를 생성하고, display()로 HTML 객체를 노트북 상에 표시한다.

In [ ]:
# JSAnimation 설치
!pip install JSAnimation

# 패키지 임포트
import matplotlib.pyplot as plt
from matplotlib import animation
from JSAnimation.IPython_display import display_animation
from IPython.display import HTML

# 애니메이션 재생 정의
plt.figure(figsize = (frames[0].shape[1] / 72,0, frames[0].shape[0] / 72.0), dpi=72)
patch = plt.imshow(frames[0])
plt.axis('off')

# 애니메이션 정기 처리
def animate(i):
    patch.set_data(frames[i])
    
# 애니메이션 재생
anim = animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval==50)
HTML(anim.to_jshtml())

# 애니메이션 재생
display_frames_as_gif(frames)